In [1]:
import pandas as pd
import json
from environs import Env
from graph_functions import query_to_df,start_graph_session, start_graph_session_crashdown_w_auth
from IPython.display import Image


env = Env()
#session = start_graph_session(env) 
session = start_graph_session_crashdown_w_auth(env) 

38687


## List of all breast-related GWAS

In [2]:
query="""
MATCH (gwas:Gwas)
where toLower(gwas.trait) contains "breast"
and not(toLower(gwas.trait) contains "baby") 
and not(toLower(gwas.trait) contains "mother")
and not(toLower(gwas.trait) contains "sibling")
and not(toLower(gwas.trait) contains "mammogram")
return distinct gwas.trait, gwas.id, gwas.sample_size as SampleSize,
toInteger(gwas.ncase) as N_case, toInteger(gwas.ncontrol) as N_control, gwas.author, gwas.year, gwas.consortium
order by SampleSize desc
"""
df = query_to_df(session, query)
#df.to_csv("query_results/bc_all_related_gwas.tsv", sep="\t", index=False)
df


MATCH (gwas:Gwas)
where toLower(gwas.trait) contains "breast"
and not(toLower(gwas.trait) contains "baby") 
and not(toLower(gwas.trait) contains "mother")
and not(toLower(gwas.trait) contains "sibling")
and not(toLower(gwas.trait) contains "mammogram")
return distinct gwas.trait, gwas.id, gwas.sample_size as SampleSize,
toInteger(gwas.ncase) as N_case, toInteger(gwas.ncontrol) as N_control, gwas.author, gwas.year, gwas.consortium
order by SampleSize desc



,gwas.trait,gwas.id,SampleSize,N_case,N_control,gwas.author,gwas.year,gwas.consortium
0,Other disorders of breast,finn-a-N14_BREASTDISOTH,None,594.0,50767.0,None,2020.0,None
1,Malignant neoplasm of breast,finn-a-C3_BREAST,None,2589.0,93910.0,None,2020.0,None
2,Malignant neoplasm of breast,finn-a-C3_BREAST_3,None,2589.0,93910.0,None,2020.0,None
3,Carcinoma in situ of breast (ICD C excluded),finn-a-CD2_INSITU_BREAST_EXALLC,None,377.0,87024.0,None,2020.0,None
4,Carcinoma in situ of breast,finn-a-CD2_INSITU_BREAST,None,377.0,96122.0,None,2020.0,None
...,...,...,...,...,...,...,...,...
65,ER- Breast cancer (GWAS),ieu-a-1166,21695.0,3611.0,18084.0,Michailidou K,2015.0,BCAC
66,ER+ Breast cancer (Combined Oncoarray; iCOGS; ...,ieu-a-1127,175475.0,69501.0,105974.0,Michailidou K,2017.0,BCAC
67,Breast cancer,ebi-a-GCST004988,139274.0,76192.0,63082.0,Michailidou K,2017.0,None
68,ER- Breast cancer (Combined Oncoarray; iCOGS; ...,ieu-a-1128,127442.0,21468.0,105974.0,Michailidou K,2017.0,BCAC


## Get BC GWAS that have MR results + exclude unrelated

In [6]:
query="""
MATCH (gwas:Gwas)<-[mr:MR_EVE_MR]-(othergwas:Gwas)
where toLower(gwas.trait) contains "breast"
    and not(toLower(gwas.trait) contains "baby") 
    and not(toLower(gwas.trait) contains "mother")
    and not(toLower(gwas.trait) contains "sibling")
    and not(toLower(gwas.trait) contains "mammogram")
    and not(toLower(gwas.trait) contains "protein")
return distinct gwas.trait, gwas.id, toInteger(gwas.sample_size) as SampleSize,
toInteger(gwas.ncase) as N_case, toInteger(gwas.ncontrol) as N_control, gwas.author, gwas.year, gwas.consortium
order by gwas.id,  gwas.author, N_case desc
"""
df = query_to_df(session, query)
#df.to_csv("query_results/bc_gwas_with_mr.tsv", sep="\t", index=False)

# calulate cases % in the dataset and only keep those >2%
df['case_percent'] = df['N_case']/df['SampleSize']*100
df = df[df['case_percent'] > 2]


#df.to_csv("query_results/bc_gwas_with_mr_keep.tsv", sep="\t", index=False)
all_bc_gwas = list(df['gwas.id'])
df



MATCH (gwas:Gwas)<-[mr:MR_EVE_MR]-(othergwas:Gwas)
where toLower(gwas.trait) contains "breast"
    and not(toLower(gwas.trait) contains "baby") 
    and not(toLower(gwas.trait) contains "mother")
    and not(toLower(gwas.trait) contains "sibling")
    and not(toLower(gwas.trait) contains "mammogram")
    and not(toLower(gwas.trait) contains "protein")
return distinct gwas.trait, gwas.id, toInteger(gwas.sample_size) as SampleSize,
toInteger(gwas.ncase) as N_case, toInteger(gwas.ncontrol) as N_control, gwas.author, gwas.year, gwas.consortium
order by gwas.id,  gwas.author, N_case desc



,gwas.trait,gwas.id,SampleSize,N_case,N_control,gwas.author,gwas.year,gwas.consortium,case_percent
0,Breast cancer,ebi-a-GCST004988,139274,76192,63082,Michailidou K,2017.0,None,54.706550
1,Breast cancer,ebi-a-GCST007236,89677,46785,42892,Michailidou K,2015.0,None,52.170568
2,Breast cancer (Combined Oncoarray; iCOGS; GWAS...,ieu-a-1126,228951,122977,105974,Michailidou K,2017.0,BCAC,53.713240
3,ER+ Breast cancer (Combined Oncoarray; iCOGS; ...,ieu-a-1127,175475,69501,105974,Michailidou K,2017.0,BCAC,39.607351
4,ER- Breast cancer (Combined Oncoarray; iCOGS; ...,ieu-a-1128,127442,21468,105974,Michailidou K,2017.0,BCAC,16.845310
5,Breast cancer (Oncoarray),ieu-a-1129,106776,61282,45494,Michailidou K,2017.0,BCAC,57.393047
6,Breast cancer (iCOGS),ieu-a-1130,89677,46785,42892,Michailidou K,2017.0,BCAC,52.170568
7,Breast cancer (GWAS),ieu-a-1131,32498,14910,17588,Michailidou K,2017.0,BCAC,45.879746
8,ER+ Breast cancer (Oncoarray),ieu-a-1132,83691,38197,45494,Michailidou K,2017.0,BCAC,45.640511
9,ER+ Breast cancer (iCOGS),ieu-a-1133,69970,27078,42892,Michailidou K,2017.0,BCAC,38.699443


In [3]:
all_bc_gwas

['ebi-a-GCST004988',
 'ebi-a-GCST007236',
 'ieu-a-1126',
 'ieu-a-1127',
 'ieu-a-1128',
 'ieu-a-1129',
 'ieu-a-1130',
 'ieu-a-1131',
 'ieu-a-1132',
 'ieu-a-1133',
 'ieu-a-1134',
 'ieu-a-1135',
 'ieu-a-1136',
 'ieu-a-1137',
 'ieu-a-1160',
 'ieu-a-1161',
 'ieu-a-1162',
 'ieu-a-1163',
 'ieu-a-1164',
 'ieu-a-1165',
 'ieu-a-1166',
 'ieu-a-1167',
 'ieu-a-1168',
 'ukb-a-55',
 'ukb-b-16890',
 'ukb-d-C3_BREAST_3']

In [7]:
# ebi-a-GCST007236 data is messed up - exclude
all_bc_gwas.remove('ebi-a-GCST007236')
all_bc_gwas.remove('ebi-a-GCST004988')
print(all_bc_gwas)

['ieu-a-1126', 'ieu-a-1127', 'ieu-a-1128', 'ieu-a-1129', 'ieu-a-1130', 'ieu-a-1131', 'ieu-a-1132', 'ieu-a-1133', 'ieu-a-1134', 'ieu-a-1135', 'ieu-a-1136', 'ieu-a-1137', 'ieu-a-1160', 'ieu-a-1161', 'ieu-a-1162', 'ieu-a-1163', 'ieu-a-1164', 'ieu-a-1165', 'ieu-a-1166', 'ieu-a-1167', 'ieu-a-1168', 'ukb-a-55', 'ukb-b-16890', 'ukb-d-C3_BREAST_3']


## Review MR data for the main dataset

In [19]:
# MR test
query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id = "ieu-a-1126"  
AND (not (toLower(exposure.trait) contains 'breast')) 
AND mr.pval < 1e-05
RETURN exposure {.id, .trait}, outcome.id, mr {.b, .pval, .method, .selection, .moescore} ORDER BY mr.pval
"""
df = query_to_df(session, query)
df


MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id = "ieu-a-1126"  
AND (not (toLower(exposure.trait) contains 'breast')) 
AND mr.pval < 1e-05
RETURN exposure {.id, .trait}, outcome.id, mr {.b, .pval, .method, .selection, .moescore} ORDER BY mr.pval



,outcome.id,exposure.trait,exposure.id,mr.b,mr.method,mr.selection,mr.pval,mr.moescore
0,ieu-a-1126,Ever addicted to alcohol,ukb-d-20406,-0.054825,FE IVW,DF,0.000000e+00,1.00
1,ieu-a-1126,Diagnoses - secondary ICD10: K44.9 Diaphragmat...,ukb-b-3672,-1.542995,FE IVW,DF,4.514856e-242,1.00
2,ieu-a-1126,Killer cell immunoglobulin-like receptor 2DL5A,prot-a-1644,0.016679,FE IVW,DF,4.770365e-106,1.00
3,ieu-a-1126,Ever had hysterectomy (womb removed),ukb-b-6445,-1.562231,FE IVW,DF,9.686474e-86,1.00
4,ieu-a-1126,Operation code: bunion/hallus valgus surgery,ukb-b-9190,1.109370,FE IVW,DF,3.334299e-85,1.00
5,ieu-a-1126,"Vascular endothelial growth factor A, isoform 121",prot-a-3197,0.013410,FE IVW,DF,7.992747e-83,1.00
6,ieu-a-1126,fatty acid binding protein 4,prot-b-71,0.019159,FE IVW,DF,1.571455e-58,1.00
7,ieu-a-1126,Diagnoses - secondary ICD10: Z92.1 Personal hi...,ukb-b-7352,-0.973106,FE IVW,DF,1.267170e-57,1.00
8,ieu-a-1126,CUB and zona pellucida-like domain-containing ...,prot-a-735,0.021479,FE IVW,DF,2.941390e-37,1.00
9,ieu-a-1126,Filamin-A,prot-a-1117,-0.054938,FE IVW,DF,9.719336e-26,1.00


In [8]:
# NB takes a long time
# MR
# get traits that have pval < 1e-05

query = """
    MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
    WHERE outcome.id in {gwas_list} 
    AND  not exposure.id  in {gwas_list}
    AND (not (toLower(exposure.trait) contains 'breast')) 
    AND mr.pval < 1e-05
    with mr, exposure, outcome
    ORDER BY mr.pval 
    RETURN exposure.id, exposure.trait, exposure.sample_size,
            collect(outcome.id) as outcome_ids, 
            collect(mr.pval) as MR_pvals, collect(mr.b) as MR_beta

""".format(gwas_list = all_bc_gwas)
df = query_to_df(session, query, True)

df


    MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
    WHERE outcome.id in ['ieu-a-1126', 'ieu-a-1127', 'ieu-a-1128', 'ieu-a-1129', 'ieu-a-1130', 'ieu-a-1131', 'ieu-a-1132', 'ieu-a-1133', 'ieu-a-1134', 'ieu-a-1135', 'ieu-a-1136', 'ieu-a-1137', 'ieu-a-1160', 'ieu-a-1161', 'ieu-a-1162', 'ieu-a-1163', 'ieu-a-1164', 'ieu-a-1165', 'ieu-a-1166', 'ieu-a-1167', 'ieu-a-1168', 'ukb-a-55', 'ukb-b-16890', 'ukb-d-C3_BREAST_3'] 
    AND  not exposure.id  in ['ieu-a-1126', 'ieu-a-1127', 'ieu-a-1128', 'ieu-a-1129', 'ieu-a-1130', 'ieu-a-1131', 'ieu-a-1132', 'ieu-a-1133', 'ieu-a-1134', 'ieu-a-1135', 'ieu-a-1136', 'ieu-a-1137', 'ieu-a-1160', 'ieu-a-1161', 'ieu-a-1162', 'ieu-a-1163', 'ieu-a-1164', 'ieu-a-1165', 'ieu-a-1166', 'ieu-a-1167', 'ieu-a-1168', 'ukb-a-55', 'ukb-b-16890', 'ukb-d-C3_BREAST_3']
    AND (not (toLower(exposure.trait) contains 'breast')) 
    AND mr.pval < 1e-05
    with mr, exposure, outcome
    ORDER BY mr.pval 
    RETURN exposure.id, exposure.trait, exposure.sample_size,
    

,exposure.id,exposure.trait,exposure.sample_size,outcome_ids,MR_pvals,MR_beta
0,prot-a-1411,Alpha-L-iduronidase,3301.0,"[ukb-b-16890, ieu-a-1136, ieu-a-1126, ieu-a-11...","[0.0, 0.0, 4.12082678662007e-25, 1.62953109690...","[0.000454060802980906, 0.0167251457326905, 0.0..."
1,prot-a-1276,"Glutamate receptor ionotropic, delta-2",3301.0,[ukb-b-16890],[0.0],[0.00273122085407538]
2,prot-a-2166,Platelet-activating factor acetylhydrolase IB ...,3301.0,[ukb-b-16890],[0.0],[-0.000697314107341724]
3,prot-a-995,Protein eva-1 homolog C,3301.0,"[ukb-b-16890, ieu-a-1132, ieu-a-1127]","[0.0, 1.71005705410616e-33, 9.05550305130723e-11]","[0.00111999698219523, -0.0269001814975584, -0...."
4,prot-a-566,Creatine kinase M-type,3301.0,[ukb-b-16890],[0.0],[-0.0020934261149299]
...,...,...,...,...,...,...
737,prot-a-821,Dickkopf-related protein 1,3301.0,[ieu-a-1136],[8.24099097862065e-06],[0.0339189126254016]
738,ukb-b-7376,Impedance of leg (right),454863.0,[ieu-a-1126],[8.29773347622307e-06],[0.105088281390953]
739,ukb-b-15782,"logMAR, final (right)",101023.0,[ukb-a-55],[8.52893962249346e-06],[0.00588635442005368]
740,ieu-a-812,Parkinson's disease,5691.0,[ieu-a-1126],[8.64938636830279e-06],[0.0588601942842277]


In [9]:
df.shape

(742, 6)

In [10]:
# now those traits that appered at least once in something at <1e05, 
# get MR results for those traits with all BC datasets

query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in {gwas_list} 
AND  exposure.id  in {exposure_list}
with mr, exposure, outcome
ORDER BY exposure.trait
RETURN exposure.id, exposure.trait, exposure.sample_size, exposure.sex, exposure.note,
toInteger(exposure.year) as year, exposure.author as author, exposure.consortium as consortium,
        outcome.id, outcome.sample_size, toInteger(outcome.ncase) as N_case, outcome.year, outcome.nsnp,
        mr.pval, mr.b, mr.se, mr.method, mr.moescore
""".format(gwas_list = all_bc_gwas,
           exposure_list = list(set(df['exposure.id'])))
df = query_to_df(session, query, True)


#df.to_csv("query_results/bc_all_mr.tsv", sep="\t", index=False)
#df.to_csv("query_results/bc_all_mr_allMR.tsv", sep="\t", index=False) with pval <1

df


MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in ['ieu-a-1126', 'ieu-a-1127', 'ieu-a-1128', 'ieu-a-1129', 'ieu-a-1130', 'ieu-a-1131', 'ieu-a-1132', 'ieu-a-1133', 'ieu-a-1134', 'ieu-a-1135', 'ieu-a-1136', 'ieu-a-1137', 'ieu-a-1160', 'ieu-a-1161', 'ieu-a-1162', 'ieu-a-1163', 'ieu-a-1164', 'ieu-a-1165', 'ieu-a-1166', 'ieu-a-1167', 'ieu-a-1168', 'ukb-a-55', 'ukb-b-16890', 'ukb-d-C3_BREAST_3'] 
AND  exposure.id  in ['ukb-b-16812', 'ieu-a-22', 'prot-a-2440', 'ubm-a-510', 'prot-a-2039', 'ukb-b-16677', 'prot-a-1288', 'ukb-b-8846', 'ubm-a-271', 'prot-a-1075', 'ieu-a-64', 'prot-a-2362', 'ukb-b-19520', 'prot-a-1833', 'ubm-a-613', 'prot-a-1930', 'ukb-a-243', 'prot-a-2557', 'prot-a-319', 'prot-a-2018', 'prot-a-212', 'ukb-b-2183', 'prot-a-3137', 'prot-a-3254', 'ukb-b-8297', 'ukb-d-G6_NERPLEX', 'ukb-b-10859', 'prot-a-1006', 'ubm-a-495', 'met-a-513', 'ukb-a-176', 'prot-a-2743', 'prot-a-2920', 'prot-a-1739', 'prot-a-2357', 'prot-a-1317', 'prot-a-213', 'prot-a-1917', 'ukb-b-9508

,exposure.id,exposure.trait,exposure.sample_size,exposure.sex,exposure.note,year,author,consortium,outcome.id,outcome.sample_size,N_case,outcome.year,outcome.nsnp,mr.pval,mr.b,mr.se,mr.method,mr.moescore
0,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ieu-a-1168,33832.0,15748,2015.0,13011123.0,8.200307e-01,-0.692852,3.045429,FE IVW,1.0
1,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ieu-a-1167,22286.0,4202,2015.0,13011123.0,5.000000e-01,-0.462777,-0.462777,Wald ratio,1.0
2,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ieu-a-1166,21695.0,3611,2015.0,13011123.0,1.253699e-02,2.923434,1.170939,FE IVW,1.0
3,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ieu-a-1165,37954.0,2900,2015.0,12940150.0,5.000000e-01,0.124119,0.124119,Wald ratio,1.0
4,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ieu-a-1164,23059.0,1333,2015.0,8714606.0,5.000000e-01,-17.286818,-17.286818,Wald ratio,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16180,ubm-a-2656,volume Left-Lateral-Ventricle,8411.0,Males,None,2018,Elliott LT,None,ieu-a-1130,89677.0,46785,2017.0,10680257.0,1.543810e-02,0.108434,0.044772,FE IVW,1.0
16181,ubm-a-2656,volume Left-Lateral-Ventricle,8411.0,Males,None,2018,Elliott LT,None,ieu-a-1129,106776.0,61282,2017.0,10680257.0,1.285261e-90,-0.158745,0.007864,FE IVW,1.0
16182,ubm-a-2656,volume Left-Lateral-Ventricle,8411.0,Males,None,2018,Elliott LT,None,ieu-a-1128,127442.0,21468,2017.0,10680257.0,2.196088e-01,0.041780,0.034035,FE IVW,1.0
16183,ubm-a-2656,volume Left-Lateral-Ventricle,8411.0,Males,None,2018,Elliott LT,None,ieu-a-1127,175475.0,69501,2017.0,10680257.0,1.951276e-01,-0.094147,0.072669,FE IVW,1.0


In [11]:
df.shape

(16185, 18)

In [12]:
df.to_csv("../query_results/bc_all_mr_allMR.tsv", sep="\t", index=False)


### Extract MR results for a list of exposure GWAS


In [6]:
# read list of exp gwas (EXTRACTED IN R)
risk_factors = list(pd.read_csv("../query_results/gwas_rf_list.txt", sep = '\t')['id'])


query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in {gwas_list_outcome} 
AND  exposure.id  in {gwas_list_exposure}
//AND mr.pval < 1e-05
with mr, exposure, outcome
ORDER BY exposure.trait
RETURN exposure.id, exposure.trait, exposure.sample_size, exposure.sex, exposure.note,
toInteger(exposure.year) as year, exposure.author as author, exposure.consortium as consortium,
        outcome.id, outcome.sample_size, toInteger(outcome.ncase) as N_case, outcome.year, outcome.nsnp,
        mr.pval, mr.b, mr.se, mr.method, mr.moescore
        
""".format(gwas_list_outcome = all_bc_gwas,
           gwas_list_exposure = risk_factors )
df = query_to_df(session, query, True)


#df.to_csv("query_results/gwas_rf_bc_all_mr.tsv", sep="\t", index=False)
df


MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in None 
AND  exposure.id  in ['ukb-b-2115', 'ukb-b-19885', 'ukb-b-5764', 'ebi-a-GCST006100', 'ukb-a-511', 'ukb-a-4', 'ukb-b-4512', 'ukb-b-15869', 'ukb-b-8865', 'ukb-b-7663', 'ukb-b-6011', 'ukb-a-507', 'ukb-b-16998', 'ukb-b-2993', 'ukb-a-483', 'ukb-b-8764', 'ukb-b-4710', 'ukb-b-3502', 'ukb-b-151', 'ukb-a-485', 'ukb-b-13184', 'ukb-a-512', 'ebi-a-GCST006097', 'ukb-b-13702', 'ukb-a-486', 'ukb-b-12205', 'ukb-b-11495', 'ukb-a-509', 'ukb-a-508', 'ukb-a-349', 'ukb-a-350', 'ukb-a-1', 'ukb-b-7337', 'ukb-b-2346', 'ukb-b-14649', 'ukb-b-1046', 'ukb-a-396', 'ukb-b-18176', 'ukb-b-13932', 'ebi-a-GCST006098', 'ukb-a-484', 'ebi-a-GCST006099', 'ukb-b-9991', 'ukb-a-482', 'ukb-a-487', 'ukb-b-2634', 'ukb-a-312', 'ukb-b-9433', 'ukb-a-315', 'ukb-a-323', 'ukb-b-9487', 'ukb-b-8727', 'ukb-b-18105', 'ukb-b-12405', 'ukb-b-9509', 'ukb-b-3768', 'ukb-b-4564', 'ukb-b-4193', 'ukb-b-9547', 'ukb-b-17422', 'ieu-a-1004', 'ukb-a-322', 'ukb-a-319', 'ukb-

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Variable `None` not defined (line 3, column 21 (offset: 74))
"WHERE outcome.id in None"
                     ^}

### Traits vs traits -- looking for intermediates

In [6]:
ids = list(pd.read_csv("../other_text_files/potential_intem_traits.csv")['x'])



query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in {ids} 
AND  exposure.id  in {ids}
AND mr.pval < 1e-05
with mr, exposure, outcome
ORDER BY exposure.trait
RETURN exposure.id, exposure.trait, exposure.sample_size, exposure.sex, exposure.note,
toInteger(exposure.year) as year, exposure.author as author, exposure.consortium as consortium,
        outcome.id, outcome.sample_size, toInteger(outcome.ncase) as N_case, outcome.year, outcome.nsnp,
        mr.pval, mr.b, mr.se, mr.method, mr.moescore
        
""".format(ids = ids )

df = query_to_df(session, query, True)


df




MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in ['ukb-d-M13_POLYARTHROPATHIES', 'met-a-558', 'met-a-682', 'met-a-427', 'met-a-500', 'ukb-b-14067', 'ukb-b-16677', 'ukb-b-1004', 'ukb-b-18436', 'ukb-b-4643', 'ukb-a-320', 'ukb-b-17422', 'prot-a-67', 'ukb-d-20117_2', 'prot-a-1154', 'prot-a-1411', 'ieu-a-298', 'prot-a-99', 'ukb-d-30630_irnt', 'ukb-d-30630_raw', 'prot-a-310', 'ukb-b-5716', 'ukb-a-30', 'ukb-b-1707', 'prot-a-216', 'prot-a-212', 'prot-a-674', 'met-a-362', 'ieu-a-974', 'met-a-656', 'prot-a-390', 'prot-a-392', 'prot-a-399', 'prot-a-408', 'ieu-a-1015', 'prot-a-426', 'prot-a-655', 'prot-a-735', 'prot-a-486', 'prot-a-488', 'prot-a-346', 'prot-a-354', 'prot-a-366', 'ukb-a-58', 'ukb-a-60', 'ukb-d-C_PRIMARY_LYMPHOID_HEMATOPOIETIC', 'prot-a-550', 'prot-a-643', 'prot-a-710', 'prot-a-532', 'ukb-d-1468_5', 'prot-a-412', 'ukb-b-2221', 'ukb-a-295', 'ieu-a-1096', 'prot-a-714', 'prot-a-381', 'ukb-a-34', 'prot-a-305', 'prot-a-322', 'prot-a-518', 'prot-a-521', 'ukb-d-602

,exposure.id,exposure.trait,exposure.sample_size,exposure.sex,exposure.note,year,author,consortium,outcome.id,outcome.sample_size,N_case,outcome.year,outcome.nsnp,mr.pval,mr.b,mr.se,mr.method,mr.moescore
0,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ukb-b-3672,463010.0,14326.0,2018.0,9851867.0,1.086372e-12,0.353187,0.049611,FE IVW,1.0
1,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ukb-d-D17,361194.0,4314.0,2018.0,11857594.0,2.784593e-10,0.204733,0.032444,FE IVW,1.0
2,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,prot-a-714,3301.0,NaN,2018.0,10534735.0,7.432935e-10,5.031430,0.817243,FE IVW,1.0
3,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ukb-d-J45,361194.0,1693.0,2018.0,10336532.0,2.094952e-11,0.270787,0.040421,FE IVW,1.0
4,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ukb-b-6445,221697.0,18171.0,2018.0,9851867.0,1.599454e-09,1.150836,0.190725,FE IVW,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4961,prot-a-2575,"rRNA methyltransferase 3, mitochondrial",3301.0,Males and Females,None,2018,Sun BB,None,prot-a-2189,3301.0,NaN,2018.0,10534735.0,2.174772e-16,0.296702,0.036130,FE IVW,1.0
4962,prot-a-2575,"rRNA methyltransferase 3, mitochondrial",3301.0,Males and Females,None,2018,Sun BB,None,prot-a-310,3301.0,NaN,2018.0,10534735.0,1.030570e-10,0.244003,0.037757,FE IVW,1.0
4963,prot-a-2575,"rRNA methyltransferase 3, mitochondrial",3301.0,Males and Females,None,2018,Sun BB,None,prot-a-2820,3301.0,NaN,2018.0,10534735.0,1.729263e-06,-0.052297,0.010935,FE IVW,1.0
4964,prot-a-2575,"rRNA methyltransferase 3, mitochondrial",3301.0,Males and Females,None,2018,Sun BB,None,prot-a-2247,3301.0,NaN,2018.0,10534735.0,1.544343e-06,-0.101734,0.021171,FE IVW,1.0


In [8]:
df.to_csv("../query_results/bc_risk_factors_mr.tsv", sep="\t", index=False)


### Correlation 

In [7]:
# find which traits are highly correlated with UKB cancers

query = """
MATCH (trait:Gwas)-[gen_cor:GEN_COR]-(assoc_trait:Gwas)
where assoc_trait.id in ['ukb-a-519']
match (trait:Gwas)-[mr:MR_EVE_MR]->(assoc_trait:Gwas)
//where mr.pval < 1e-03
return distinct trait.trait, trait.id, gen_cor.rg, assoc_trait.trait, assoc_trait.id, mr.b, mr.pval
order by gen_cor.rg desc
"""
df = query_to_df(session, query)
df





MATCH (trait:Gwas)-[gen_cor:GEN_COR]-(assoc_trait:Gwas)
where assoc_trait.id in ['ukb-a-519']
match (trait:Gwas)-[mr:MR_EVE_MR]->(assoc_trait:Gwas)
//where mr.pval < 1e-03
return distinct trait.trait, trait.id, gen_cor.rg, assoc_trait.trait, assoc_trait.id, mr.b, mr.pval
order by gen_cor.rg desc



,trait.trait,trait.id,gen_cor.rg,assoc_trait.trait,assoc_trait.id,mr.b,mr.pval
0,Number of self-reported cancers,ukb-a-21,0.8279,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,0.077594,1.850559e-02
1,Had other major operations,ukb-a-326,0.2660,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,0.026946,5.000000e-01
2,Breastfed as a baby,ukb-a-33,0.2471,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.012162,5.060810e-01
3,Headaches for 3+ months,ukb-a-354,0.1980,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.017729,3.182665e-29
4,Qualifications: NVQ or HND or HNC or equivalent,ukb-a-402,0.1885,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,0.026366,5.000000e-01
...,...,...,...,...,...,...,...
260,Reason for glasses/contact lenses: For 'astigm...,ukb-a-422,-0.1279,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.077385,5.000000e-01
261,Diagnoses - main ICD10: I21 Acute myocardial i...,ukb-a-533,-0.1304,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.057651,5.647018e-01
262,Number of days/week of vigorous physical activ...,ukb-a-511,-0.1306,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.006864,3.821609e-02
263,Mineral and other dietary supplements: None of...,ukb-a-493,-0.1405,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.014608,6.539805e-01
